# Load Data

In [1]:
from pyearth import Earth
from DLtools.Data import instant_data,check_specific_col,station_sel,del_less_col

import numpy as np
import pandas as pd

In [2]:
def to_supervise(data,target,n_out):
    data[target]=data[target].shift(-n_out)
    data = data.astype('float64').dropna()
    X = data.drop([target],axis=1)
    xlabels = list(X.columns)

    X = X.values
    y = data[target].values
    return X,y,xlabels

In [3]:
loading = instant_data()
df,mode = loading.hourly_instant(),'hour'
# df,mode = loading.daily_instant(),'day'

if mode =='hour': n_past,n_future = 24*7,72
elif mode =='day': n_past,n_future = 30,14
else: n_future=None; print('incorrect input')

st = 'CPY012'
target,start_p,stop_p,host_path=station_sel(st,mode)

#start_p,stop_p = '2016-01-01','2018-01-01'

save_path = '/home/song/Public/Song/Work/Thesis/MAR_4(fulldate)/'

In [4]:
data = df[start_p:stop_p]
data = del_less_col(data,ratio=.9)
data = data.interpolate(limit=3000000000,limit_direction='both').astype('float32')
data['Day'] = data.index.dayofyear #add day
data.shape


Before del col are... 337


/home/song/anaconda3/envs/tf/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


After... 95


(24600, 96)

In [5]:
# ## Shift target station to future 
n_out = n_future
data[target]=data[target].shift(-n_out)
data = data.astype('float32').dropna()

X = data.drop([target],axis=1)
xlabels = list(X.columns)


# # # REPLACE NAN WITH 0
# # X = X.fillna(0).values
# # y = np.log(data[TARGET].fillna(0).values)
# # where_are_NaNs = np.isnan(y)
# # y[where_are_NaNs] = 0
X=X.values
y=data[target].values

# MARS feature selection (from all stations)

In [15]:
#Fit an Earth model
criteria = ('rss', 'gcv', 'nb_subsets')
model = Earth(enable_pruning = True,
            #   max_degree=3,
            #   max_terms=10,
              minspan_alpha=.5,
              feature_importance_type=criteria,
              verbose=True)
model.fit(X,y,xlabels=xlabels)


Beginning forward pass
---------------------------------------------------------------
iter  parent  var  knot  mse       terms  gcv    rsq    grsq   
---------------------------------------------------------------
0     -       -    -     0.857487  1      0.858  0.000  0.000  
1     0       11   16436  0.094666  3      0.095  0.890  0.890  
2     0       42   566   0.088795  5      0.089  0.896  0.896  
3     0       39   1108  0.083857  7      0.084  0.902  0.902  
4     0       29   2873  0.079609  9      0.080  0.907  0.907  
5     0       13   3493  0.075891  11     0.076  0.911  0.911  
6     0       2    -1    0.071265  12     0.071  0.917  0.917  
7     0       23   3351  0.068508  14     0.069  0.920  0.920  
8     0       27   6993  0.066703  16     0.067  0.922  0.922  
9     0       28   11320  0.065367  18     0.066  0.924  0.923  
10    0       35   9836  0.064189  20     0.065  0.925  0.925  
11    0       10   7813  0.063139  22     0.064  0.926  0.926  
12    0       9

/home/song/anaconda3/envs/tf/lib/python3.8/site-packages/pyearth/earth.py:813: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  pruning_passer.run()


1     10  31     0.06  0.056  0.935  0.934  
2     18  30     0.06  0.056  0.935  0.934  
3     31  29     0.06  0.056  0.935  0.934  
4     25  28     0.06  0.056  0.935  0.934  
5     3   27     0.06  0.057  0.935  0.934  
6     27  26     0.06  0.057  0.935  0.934  
7     14  25     0.06  0.057  0.934  0.934  
8     20  24     0.06  0.057  0.934  0.934  
9     29  23     0.06  0.057  0.934  0.933  
10    12  22     0.06  0.057  0.934  0.933  
11    28  21     0.06  0.058  0.933  0.933  
12    19  20     0.06  0.058  0.932  0.932  
13    13  19     0.06  0.059  0.932  0.931  
14    8   18     0.06  0.059  0.931  0.931  
15    2   17     0.06  0.060  0.930  0.930  
16    30  16     0.06  0.061  0.930  0.929  
17    1   15     0.06  0.062  0.929  0.928  
18    16  14     0.06  0.062  0.928  0.927  
19    17  13     0.06  0.063  0.927  0.927  
20    15  12     0.06  0.063  0.926  0.926  
21    7   11     0.06  0.065  0.925  0.925  
22    23  10     0.07  0.066  0.923  0.923  
23    5   

/home/song/anaconda3/envs/tf/lib/python3.8/site-packages/pyearth/earth.py:1066: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coef, resid = np.linalg.lstsq(B, weighted_y[:, i])[0:2]


Earth(feature_importance_type=('rss', 'gcv', 'nb_subsets'), minspan_alpha=0.5,
      verbose=True)

In [62]:
# #Print the model
# print(model.trace())
# print(model.summary())

* Feature Importance

In [9]:
print(model.summary_feature_importances(sort_by='rss')[:700])

                                   rss    gcv    nb_subsets
CPY006_wl                          0.83   0.84   0.05          
NAN013_wl                          0.04   0.04   0.03          
KWN002_wl                          0.02   0.02   0.03          
DIV006_wl                          0.02   0.01   0.10          
Dam_SK_Useable_WaterVol            0.01   0.01   0.03          
CPY009_wl                          0.01   0.01   0.03          
CPY008_wl                          0.01   0.01   0.03          
CPY016_wl                          0.01   0.01   0.03          
CPY004_wl                          0.01   0.01   0.03          
YOM008_wl                          0.01   0.01   0.03          



In [6]:
def toDF(rank):
    name,rss,gcv,nb_sub= list(),list(),list(),list()
    for i in range(len(rank)):
        if i%4==0:
            name.append(rank[i])
        elif i%4==1:
            rss.append(float(rank[i]))
        elif i%4==2:
            gcv.append(float(rank[i]))
        elif i%4==3:
            nb_sub.append(float(rank[i]))
    data = {'feature':name,
    'rss':rss,
    'gcv':gcv,
    'nb_subset':nb_sub}
    score = pd.DataFrame(data)
    return score



In [7]:
nbsub = model.summary_feature_importances(sort_by='nb_subsets')[:2000].split()[3:83]
gcv = model.summary_feature_importances(sort_by='gcv')[:2000].split()[3:83]
rss = model.summary_feature_importances(sort_by='rss')[:2000].split()[3:83]

rss = toDF(rss)
gcv = toDF(gcv)
nbsub = toDF(nbsub)


NameError: name 'model' is not defined

In [6]:
thrd = 0.05
rss=rss.loc[rss['rss']>thrd]
gcv=gcv.loc[gcv['gcv']>thrd]
nbsub=nbsub.loc[nbsub['nb_subset']>thrd]

NameError: name 'rss' is not defined

In [45]:
#From top 20 rss,gcv,nbsub
top20 = pd.concat([rss,gcv,nbsub],ignore_index=True).drop_duplicates('feature')
top20

,feature,rss,gcv,nb_subset
0,CPY006_wl,0.83,0.84,0.05
2,DIV006_wl,0.02,0.01,0.10


In [39]:
if mode =='hour': top20.to_csv('MAR/[CPY012]featurelist_MAR_hourly.csv',index=False)
elif mode =='day': top20.to_csv('MAR/[CPY012]featurelist_MAR_daily.csv',index=False)
else: print('incorrect input')

# Loop

In [7]:
conclude_df=pd.DataFrame()
for n_out in range(1,n_future):
    X,y,xlabels = to_supervise(data,target,n_out)
    criteria = ('rss', 'gcv', 'nb_subsets')
    model = Earth(enable_pruning = True,
                   max_degree=3,
                  max_terms=20,
                minspan_alpha=.5,
                feature_importance_type=criteria,
                verbose=True)
    model.fit(X,y,xlabels=xlabels)
    # nbsub = model.summary_feature_importances(sort_by='nb_subsets')[:2000].split()[3:83]
    gcv = model.summary_feature_importances(sort_by='gcv')[:2000].split()[3:83]
    rss = model.summary_feature_importances(sort_by='rss')[:2000].split()[3:83]
    
    # rss,gcv,nbsub = toDF(rss),toDF(gcv),toDF(nbsub)
    rss,gcv = toDF(rss),toDF(gcv)
    thrd = 0.1
    rss=rss.loc[rss['rss']>thrd]
    gcv=gcv.loc[gcv['gcv']>thrd]
    #nbsub=nbsub.loc[nbsub['nb_subset']>thrd]

    top20=pd.concat([rss,gcv],ignore_index=True)
    top20 = pd.concat([rss,gcv],ignore_index=True).drop_duplicates('feature')
    top20['timestep'] = n_out

    #ADDED combine all result
    conclude_df = pd.concat([conclude_df,top20],ignore_index=True)
    # conclude_df.to_csv('/home/song/Public/Song/Work/Thesis/MAR/[CPY012]featurelist_MAR_hourly.csv',index=False)

Beginning forward pass
---------------------------------------------------------------
iter  parent  var  knot  mse       terms  gcv    rsq    grsq   
---------------------------------------------------------------
0     -       -    -     0.332181  1      0.332  0.000  0.000  
1     0       11   23023  0.125124  3      0.125  0.623  0.623  
2     0       49   4811  0.109048  5      0.109  0.672  0.671  
3     0       44   20873  0.102633  7      0.103  0.691  0.691  
4     0       21   16712  0.095594  9      0.096  0.712  0.712  
5     8       12   23843  0.090768  11     0.091  0.727  0.726  
6     4       43   11448  0.087508  13     0.088  0.737  0.736  
7     0       32   23311  0.084975  15     0.085  0.744  0.743  
8     8       54   23829  0.082310  17     0.083  0.752  0.751  
9     16      4    18224  0.080046  19     0.080  0.759  0.758  
10    3       27   2293  0.077668  21     0.078  0.766  0.765  
---------------------------------------------------------------
Stopping 

/home/song/anaconda3/envs/tf/lib/python3.8/site-packages/pyearth/earth.py:813: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  pruning_passer.run()


1     15  20     0.08  0.078  0.766  0.765  
2     19  19     0.08  0.078  0.766  0.765  
3     9   18     0.08  0.078  0.766  0.765  
4     7   17     0.08  0.078  0.766  0.765  
5     11  16     0.08  0.078  0.766  0.765  
6     16  15     0.08  0.078  0.765  0.764  
7     13  14     0.08  0.079  0.761  0.761  
8     14  13     0.08  0.081  0.757  0.756  
9     20  12     0.08  0.083  0.750  0.749  
10    18  11     0.09  0.086  0.743  0.742  
11    5   10     0.09  0.087  0.737  0.737  
12    17  9      0.09  0.089  0.734  0.733  
13    3   8      0.09  0.092  0.724  0.724  
14    6   7      0.09  0.093  0.719  0.719  
15    10  6      0.10  0.098  0.705  0.704  
16    8   5      0.10  0.104  0.688  0.687  
17    12  4      0.11  0.109  0.671  0.671  
18    4   3      0.13  0.125  0.623  0.623  
19    2   2      0.15  0.153  0.540  0.540  
20    1   1      0.33  0.332  -0.000  -0.000  
----------------------------------------------
Selected iteration: 2


/home/song/anaconda3/envs/tf/lib/python3.8/site-packages/pyearth/earth.py:1066: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coef, resid = np.linalg.lstsq(B, weighted_y[:, i])[0:2]


Beginning forward pass
---------------------------------------------------------------
iter  parent  var  knot  mse       terms  gcv    rsq    grsq   
---------------------------------------------------------------
0     -       -    -     0.332208  1      0.332  0.000  0.000  
1     0       11   23023  0.125438  3      0.125  0.622  0.622  
2     2       12   -1    0.107365  4      0.107  0.677  0.677  
3     2       21   20581  0.096388  6      0.096  0.710  0.710  
4     4       44   17890  0.090335  8      0.090  0.728  0.728  
5     0       46   18780  0.086277  10     0.086  0.740  0.740  
6     0       32   23311  0.083609  12     0.084  0.748  0.748  
7     8       27   2293  0.081941  14     0.082  0.753  0.753  
8     0       1    12050  0.080297  16     0.081  0.758  0.758  
9     15      4    18130  0.078570  18     0.079  0.763  0.763  
10    0       37   19641  0.077174  20     0.077  0.768  0.767  
11    13      43   17533  0.075812  22     0.076  0.772  0.771  
--------

...

/home/song/anaconda3/envs/tf/lib/python3.8/site-packages/pyearth/earth.py:1066: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coef, resid = np.linalg.lstsq(B, weighted_y[:, i])[0:2]


Beginning forward pass
---------------------------------------------------------------
iter  parent  var  knot  mse       terms  gcv    rsq    grsq   
---------------------------------------------------------------
0     -       -    -     0.357333  1      0.357  0.000  0.000  
1     0       52   299   0.291182  3      0.291  0.185  0.185  
2     2       38   2130  0.262004  5      0.262  0.267  0.266  
3     3       36   18389  0.214944  7      0.215  0.398  0.398  
4     4       94   -1    0.180533  8      0.181  0.495  0.494  
5     0       24   2030  0.166575  10     0.167  0.534  0.533  
6     0       12   15296  0.153857  12     0.154  0.569  0.568  
7     9       40   17947  0.144619  14     0.145  0.595  0.594  
8     0       82   2766  0.138045  16     0.139  0.614  0.612  
9     8       54   -1    0.132871  17     0.133  0.628  0.627  
10    4       52   -1    0.128103  18     0.129  0.642  0.640  
11    15      34   12530  0.125018  20     0.126  0.650  0.649  
12    4      

/home/song/anaconda3/envs/tf/lib/python3.8/site-packages/pyearth/earth.py:813: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  pruning_passer.run()


1     18  20     0.12  0.122  0.660  0.658  
2     5   19     0.12  0.122  0.659  0.658  
3     2   18     0.12  0.122  0.659  0.657  
4     10  17     0.12  0.123  0.658  0.657  
5     14  16     0.12  0.123  0.657  0.655  
6     13  15     0.12  0.124  0.654  0.653  
7     3   14     0.12  0.125  0.652  0.651  
8     20  13     0.13  0.128  0.642  0.641  
9     19  12     0.13  0.131  0.634  0.633  
10    8   11     0.14  0.136  0.621  0.620  
11    16  10     0.14  0.136  0.621  0.620  
12    17  9      0.14  0.141  0.607  0.606  
13    15  8      0.15  0.148  0.588  0.587  
14    12  7      0.16  0.156  0.564  0.564  
15    11  6      0.17  0.170  0.525  0.525  
16    4   5      0.18  0.184  0.484  0.484  
17    9   4      0.21  0.209  0.414  0.414  
18    1   3      0.25  0.254  0.291  0.290  
19    6   2      0.30  0.302  0.154  0.154  
20    7   1      0.36  0.357  -0.000  -0.000  
----------------------------------------------
Selected iteration: 1


/home/song/anaconda3/envs/tf/lib/python3.8/site-packages/pyearth/earth.py:1066: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coef, resid = np.linalg.lstsq(B, weighted_y[:, i])[0:2]


In [8]:
final = conclude_df.drop_duplicates('feature')
if mode =='hour': 
    final.to_csv('MAR_4(fulldate)/[CPY012]featurelist_MAR_hourly.csv',index=False)
    conclude_df.to_csv('MAR_3/[CPY012]featurelist_MAR_hourly_Fullreport.csv',index=False)
elif mode =='day': 
    final.to_csv('MAR/[CPY012]featurelist_MAR_daily.csv',index=False)
    conclude_df.to_csv('MAR/[CPY012]featurelist_MAR_daily_Fullreport.csv',index=False)
else: print('incorrect input')

In [9]:
conclude_df.loc[conclude_df['timestep']<=72]

,feature,rss,gcv,nb_subset,timestep
0,CPY006_wl,0.75,0.75,0.07,1
1,CPY006_wl,0.73,0.73,0.21,2
2,GLF001_wl,0.15,0.15,0.12,2
3,CPY006_wl,0.69,0.69,0.12,3
4,CPY015_wl,0.24,0.24,0.21,3
...,...,...,...,...,...
149,DNP028_press,0.39,0.39,0.19,70
150,DIV006_wl,0.27,0.27,0.13,70
151,DNP028_press,0.36,0.36,0.26,71
152,YOM012_wl,0.26,0.26,0.21,71


In [10]:
set(conclude_df['feature'])

{'BKK002_wl',
 'BKK013_wl',
 'BKK021_wl',
 'CHM002_wl',
 'CPY003_wl',
 'CPY006_wl',
 'CPY015_wl',
 'DIV003_wl',
 'DIV006_wl',
 'DNP006_press',
 'DNP006_wl',
 'DNP007_humid',
 'DNP025_temp',
 'DNP028_press',
 'Day',
 'GLF001_wl',
 'KWN002_wl',
 'NAN002_wl',
 'NAN005_wl',
 'NAN009_wl',
 'NAN012_wl',
 'NAN013_wl',
 'PAS001_wl',
 'PAS006_wl',
 'PIN004_wl',
 'WAN003_wl',
 'WAN005_wl',
 'YOM012_wl'}

In [11]:
set(conclude_df['feature'].loc[(conclude_df['rss']>=.3)|(conclude_df['gcv']>=.3)])

{'BKK013_wl',
 'BKK021_wl',
 'CPY006_wl',
 'DNP028_press',
 'NAN009_wl',
 'PIN004_wl',
 'WAN003_wl',
 'WAN005_wl',
 'YOM012_wl'}

In [12]:
set(conclude_df['feature'].loc[(conclude_df['rss']>=.2)|(conclude_df['gcv']>=.2)])

{'BKK013_wl',
 'BKK021_wl',
 'CPY003_wl',
 'CPY006_wl',
 'CPY015_wl',
 'DIV003_wl',
 'DIV006_wl',
 'DNP006_wl',
 'DNP028_press',
 'NAN009_wl',
 'PAS006_wl',
 'PIN004_wl',
 'WAN003_wl',
 'WAN005_wl',
 'YOM012_wl'}